In [1]:
import os
import pandas as pd
import numpy as np

os.listdir()

['topspin_player_stats_yearly.csv',
 '.ipynb_checkpoints',
 'train.ipynb',
 'topspin_game.csv']

In [2]:
game = pd.read_csv('topspin_game.csv')
players = pd.read_csv('topspin_player_stats_yearly.csv')

/usr/local/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (3,4,7,8) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
players

,id,year,nmatches,age,minutes,ace,df,svpt,1stIn,1stWon,2ndWon,SvGms,bpSaved,bpFaced
0,0,2011,1,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,2013,1,17.6,59.0,6.0,7.0,47.0,24.0,18.0,5.0,8.0,6.0,11.0
2,0,2014,5,18.7,205.0,11.0,4.0,146.0,80.0,56.0,37.0,24.0,4.0,10.0
3,0,2015,19,19.7,249.0,44.0,30.0,661.0,370.0,257.0,137.0,99.0,41.0,69.0
4,0,2016,84,20.7,6880.0,434.0,216.0,5526.0,3230.0,2364.0,1160.0,866.0,280.0,452.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146033,41088,1968,3,41.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
146034,41103,1980,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
146035,41103,1982,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
146036,41104,1981,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df = []
for year in range(min(players['year']), max(players['year'])):
    x = players[players['year'] < year] # non-inclusive!
    x = x.groupby('id').sum().reset_index()
    x = x[x['nmatches'] > 10] # filter pointless players
    x['year'] = year
    df.append(x)
df = pd.concat(df)
feat_cols = ['ace', 'df', 'svpt', '1stIn', '1stWon', '2ndWon', 'SvGms', 'bpSaved', 'bpFaced']
for feat in feat_cols:
    df[feat] /= df['nmatches']
df = df[feat_cols+['id', 'year']]
df

,ace,df,svpt,1stIn,1stWon,2ndWon,SvGms,bpSaved,bpFaced,id,year
47,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20841,1889
47,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20841,1890
51,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20845,1890
62,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20856,1890
47,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20841,1891
...,...,...,...,...,...,...,...,...,...,...,...
36589,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40520,2022
36600,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40550,2022
36603,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40553,2022
36608,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40558,2022


In [5]:
game['year'] = game['tourney_id'].apply(lambda x: int(x.split('-')[0]))
tmp = df.merge(game, left_on=['id', 'year'], right_on=['winner_id', 'year'])
print(tmp.columns)
tmp = df.merge(tmp, left_on=['id', 'year'], right_on=['loser_id', 'year'])
print(tmp.columns)

Index(['ace', 'df', 'svpt', '1stIn', '1stWon', '2ndWon', 'SvGms', 'bpSaved',
       'bpFaced', 'id', 'year', 'tourney_id', 'match_num', 'winner_id',
       'winner_seed', 'winner_entry', 'winner_age', 'loser_id', 'loser_seed',
       'loser_entry', 'loser_age', 'score', 'max_sets', 'round', 'minutes',
       'w_ace', 'w_df', 'w_svpt', 'w_1stIn', 'w_1stWon', 'w_2ndWon', 'w_SvGms',
       'w_bpSaved', 'w_bpFaced', 'l_ace', 'l_df', 'l_svpt', 'l_1stIn',
       'l_1stWon', 'l_2ndWon', 'l_SvGms', 'l_bpSaved', 'l_bpFaced',
       'winner_rank', 'winner_rank_points', 'loser_rank', 'loser_rank_points'],
      dtype='object')
Index(['ace_x', 'df_x', 'svpt_x', '1stIn_x', '1stWon_x', '2ndWon_x', 'SvGms_x',
       'bpSaved_x', 'bpFaced_x', 'id_x', 'year', 'ace_y', 'df_y', 'svpt_y',
       '1stIn_y', '1stWon_y', '2ndWon_y', 'SvGms_y', 'bpSaved_y', 'bpFaced_y',
       'id_y', 'tourney_id', 'match_num', 'winner_id', 'winner_seed',
       'winner_entry', 'winner_age', 'loser_id', 'loser_seed', 'loser_e

In [6]:
feats = np.array(tmp[[x+'_x' for x in feat_cols]+[x+'_y' for x in feat_cols]])
feats = feats[np.sum(feats, axis=1) > 0] # remove unnecessary data

N, d = feats.shape

labels = np.ones((N,))

# randomly reverse the ordering for half the data
ind = np.random.permutation(N)[:N//2]
tmp_data = feats[ind, :d//2]
feats[ind, :d//2] = feats[ind, d//2:]
feats[ind, d//2:] = tmp_data
labels[ind] = 0

# split shuffled data into train and test
ind = np.random.permutation(N)
feats, labels = feats[ind], labels[ind]
trn_tst = int(N*0.70)
trn_feats, trn_labels = feats[:trn_tst], labels[:trn_tst]
tst_feats, tst_labels = feats[trn_tst:], labels[trn_tst:]

In [7]:
np.mean(tst_labels == 1) # random guessing baseline!

0.5003591250408097

In [8]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

clf = make_pipeline(StandardScaler(), SVC(gamma='auto', verbose=True))
clf.fit(trn_feats, trn_labels)

[LibSVM]..........................................................................................................................................................................................
*...........................*..
*
optimization finished, #iter = 214370
obj = -233841.181357, rho = 0.254423
nSV = 235453, nBSV = 234465
Total nSV = 235453


Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(gamma='auto', verbose=True))])

# vegas odds: 68.91 % accuracy

In [9]:
trn_pred = clf.predict(trn_feats)
tst_pred = clf.predict(tst_feats)

print('TRN ACC:', np.mean(trn_pred == trn_labels))
print('TST ACC:', np.mean(tst_pred == tst_labels))

TRN ACC: 0.6081475314644709
TST ACC: 0.6082843617368593


# V1 model: 60.82% accuracy

SVM with all 18 feats and entire career stats

In [10]:
import pickle
with open('model_v1.pkl', 'wb') as f:
    pickle.dump(clf, f)